## Alterações a serem feitas:
+ Criar tag de inicio e fim de arquivo
+ Analisar o formato dos dados que tem maior acerto e menor acerto tambem.

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


os.chdir("mock")

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*/**/***/****.{}'.format(extension))]

In [3]:
all_filenames[0]

'161704902/[PRATICA_ETAPA_1]/Documentos/20180510_Rcl_22328_314302526.ner.csv'

In [4]:
# Concatena todos os arquivos csv encontrados dentro de 'mock'
combined_csv = pd.concat([pd.read_csv(f,delimiter=';', na_values='NaN') for f in all_filenames])
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

# Criar uma tag inicio e fim de arquivo

In [5]:
# Cria uma coluna para cada arquivo csv identificando cada um e appenda esses dataframes
frames = []
for all_files in all_filenames:
    df = pd.read_csv(all_files,delimiter=';', na_values='NaN')
    df['file'] = all_files.split("/")[-1]
    frames.append(df)
combined_csv = pd.concat(frames)
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')
# len(df1.file.unique())

In [16]:
# list(frames[0].Token.values)

In [6]:
combined_csv.head()

,Token,Tag,file
0,Ementa,O,20180510_Rcl_22328_314302526.ner.csv
1,,O,20180510_Rcl_22328_314302526.ner.csv
2,e,O,20180510_Rcl_22328_314302526.ner.csv
3,,O,20180510_Rcl_22328_314302526.ner.csv
4,Acórdão,O,20180510_Rcl_22328_314302526.ner.csv


In [8]:
# Erro de memoria com todas as linhas

# combined_csv = combined_csv
X = combined_csv.drop('Tag', axis=1) # Define o conjunto X
v = DictVectorizer(sparse=True) # Função que transforma listas de features em vetores
X = v.fit_transform(X.to_dict('records')) #Aplica a função de vetorização no conjunto 
                                          #X que foi colocado no formato 'records' (informa o que preenche cada coluna 
                                          # da linha i)
y = combined_csv.Tag.values # Define o conjunto y

classes = np.unique(y) # Define quais serão as classes baseado nos valores únicos da coluna y
classes = classes.tolist() # Tranforma as classes de array para lista

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0) # Divide o conjunto em treino
                                                                                            #e teste
X_train.shape, y_train.shape # Formato dos dados

((648499, 26575), (648499,))

In [9]:
classes

['B_Doutrina',
 'B_Doutrinador',
 'B_Pessoa',
 'B_Precedente',
 'B_Ref. Legislativa',
 'I_Doutrina',
 'I_Doutrinador',
 'I_Pessoa',
 'I_Precedente',
 'I_Ref. Legislativa',
 'O']

In [10]:
def word2features(doc, i):
#     word = doc.iloc[i][0]
#     postag = doc.iloc[i][1]
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
#         word1 = doc.iloc[i+1][0]
#         postag1 = doc.iloc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features
# Agrupar arquivos

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
#     return [label for token, label, postag in sent]
    return [label for token, label in sent]
def sent2tokens(sent):
#     return [token for token, label, postag in sent]
    return [token for token, label in sent]

In [17]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [18]:
# Removendo a tag 'O'
new_classes = classes.copy()
new_classes.pop()
new_classes

['B_Doutrina',
 'B_Doutrinador',
 'B_Pessoa',
 'B_Precedente',
 'B_Ref. Legislativa',
 'I_Doutrina',
 'I_Doutrinador',
 'I_Pessoa',
 'I_Precedente',
 'I_Ref. Legislativa']

In [19]:
# Separa as frases para criar contexto na aprendizagem

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
#         agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
#                                                            s['POS'].values.tolist(), 
#                                                            s['Tag'].values.tolist())]
        agg_func = lambda s: [(w, t) for w, t in zip(s['Token'].values.tolist(),
                                                     s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Token').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

In [20]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [21]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    all_possible_transitions=True)
crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=new_classes)

/home/kaline/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/kaline/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.9989149305555556